# Import Necessary Libraby

In [1]:
import pandas as pd
import numpy as np
import nltk
nltk.download('punkt')
import re
from nltk.corpus import stopwords

[nltk_data] Downloading package punkt to C:\Users\Atharva
[nltk_data]     Ingale\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


# Load the Article

In [2]:
df = pd.read_csv("F:/ETL/Dataset/tennis.txt")
df.head()

,article_id,article_text,source
0,1,Maria Sharapova has basically no friends as te...,https://www.tennisworldusa.org/tennis/news/Mar...
1,2,"BASEL, Switzerland (AP), Roger Federer advance...",http://www.tennis.com/pro-game/2018/10/copil-s...
2,3,Roger Federer has revealed that organisers of ...,https://scroll.in/field/899938/tennis-roger-fe...
3,4,Kei Nishikori will try to end his long losing ...,http://www.tennis.com/pro-game/2018/10/nishiko...
4,5,"Federer, 37, first broke through on tour over ...",https://www.express.co.uk/sport/tennis/1036101...


In [3]:
df['article_text'][4]

'Federer, 37, first broke through on tour over two decades ago and he has since gone on to enjoy a glittering career. The 20-time Grand Slam winner is chasing his 99th ATP title at the Swiss Indoors this week and he faces Jan-Lennard Struff in the second round on Thursday (6pm BST). Davenport enjoyed most of her success in the late 1990s and her third and final major tournament win came at the 2000 Australian Open. But she claims the mentality of professional tennis players slowly began to change after the new millennium. "It seems pretty friendly right now," said Davenport. "I think there is a really nice environment and a great atmosphere, especially between some of the veteran players helping some of the younger players out. "It\'s a very pleasant atmosphere, I\'d have to say, around the locker rooms. "I felt like the best weeks that I had to get to know players when I was playing were the Fed Cup weeks or the Olympic weeks, not necessarily during the tournaments. "And even though m

# Tokanize the sentence and create a flatten sentence

In [4]:
from nltk.tokenize import sent_tokenize
sentences = []
for s in df['article_text']:
    sentences.append(sent_tokenize(s))

sentences = [y for x in sentences for y in x]

# Loading GloVe Word Embeddings and  Preprocessing Sentences

In [5]:
word_embeddings = {}
glove_file_path = 'F:/ETL/Dataset/GLove/glove.6B.100d.txt'
with open(glove_file_path, encoding='utf-8') as f:
    for line in f:
        values = line.split()
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        word_embeddings[word] = coefs

clean_sentences = [s.lower() for s in sentences]

# Remove non-alphabetic characters
clean_sentences = [' '.join([word for word in s.split() if word.isalpha()]) for s in clean_sentences]

# Remove stopwords
stop_words = set(stopwords.words('english'))
clean_sentences = [' '.join([word for word in s.split() if word not in stop_words]) for s in clean_sentences]


# Creating Sentence Vectors

In [6]:
sentence_vectors = []
for sentence in clean_sentences:
    words = sentence.split()
    vector_sum = np.zeros((100,))
    word_count = 0

    for word in words:
        word_vector = word_embeddings.get(word, np.zeros((100,)))
        vector_sum += word_vector
        word_count += 1

    if word_count != 0:
        sentence_vector = vector_sum / word_count
    else:
        sentence_vector = np.zeros((100,))
    
    sentence_vectors.append(sentence_vector)


#  Calculating Similarity Matrix

In [7]:
from sklearn.metrics.pairwise import cosine_similarity

# Initialize the similarity matrix
sim_mat = cosine_similarity(sentence_vectors, sentence_vectors)
np.fill_diagonal(sim_mat, 0.0)


# Ranking Sentences with PageRank and Generating Summaries

In [18]:
import networkx as nx

nx_graph = nx.from_numpy_array(sim_mat)
scores = nx.pagerank(nx_graph)

ranked_sentences = []
for i, sentence in enumerate(sentences):
    score = scores[i]
    ranked_sentences.append((score, sentence))
ranked_sentences.sort(reverse=True)

for i in range(5):
    print("ARTICLE:")
    print(df['article_text'][i])
    print('\n')
    print("SUMMARY:")
    print(ranked_sentences[i][1])
    print('\n')



ARTICLE:
Maria Sharapova has basically no friends as tennis players on the WTA Tour. The Russian player has no problems in openly speaking about it and in a recent interview she said: 'I don't really hide any feelings too much. I think everyone knows this is my job here. When I'm on the courts or when I'm on the court playing, I'm a competitor and I want to beat every single person whether they're in the locker room or across the net.So I'm not the one to strike up a conversation about the weather and know that in the next few minutes I have to go and try to win a tennis match. I'm a pretty competitive girl. I say my hellos, but I'm not sending any players flowers as well. Uhm, I'm not really friendly or close to many players. I have not a lot of friends away from the courts.' When she said she is not really close to a lot of players, is that something strategic that she is doing? Is it different on the men's tour than the women's tour? 'No, not at all. I think just because you're in t